In [1]:
import torch
import numpy as np
import pandas as pd
from collections import namedtuple
from dataclasses import dataclass
from typing import List, Tuple, Dict, Callable, Any
from board import ConnectFourField
from env import Env

In [ ]:
env = 